<a href="https://www.kaggle.com/code/swish9/multi-class-classification-of-fruits-using-tf?scriptVersionId=143165484" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.initializers import GlorotNormal

In [ ]:
train_dir = '/kaggle/input/fruits-dataset-images/images'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(9, activation='softmax')
])

**I tried using He and Glorot Initialization techniques for weights, but I was getting accuracies around 0.6 to 0.7. However, with random initialization, my highest accuracy was 0.84**

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
lr_sch = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
num_epochs = 100
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    verbose=1,
    callbacks=[lr_sch]
)

In [ ]:
train_loss, train_accuracy = model.evaluate(train_generator, steps=len(train_generator))

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")

In [ ]:
model.save('/kaggle/working/myFruitclassifier001')  

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_fruit_class(image_path, model_path='/kaggle/working/myFruitclassifier001'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array /= 255.0 
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    class_labels = ['apple fruit', 'banana fruit', 'cherry fruit', 'chickoo fruit', 'grapes fruit', 'kiwi fruit', 'mango fruit', 'orange fruit', 'strawberry fruit']
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label



In [ ]:
predict_fruit_class('/kaggle/input/strawberry/three-strawberries-strawberry-leaf-white-background-114284301.jpg.webp')

In [ ]:
predict_fruit_class('/kaggle/input/validation-banana/Banana-Single.jpg')

In [ ]:
predict_fruit_class('/kaggle/input/chickoo/1T_qlJrY6rGawO_FRcy3q9F9rC1DyB693.png.webp')